In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## EDA for articles.csv

In [2]:
articles_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv")

In [68]:
articles_df.head()

,article_id,prod_name,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name,detail_desc
0,108775015,Strap top,Vest top,Garment Upper body,Solid,Black,Dark,Black,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,Strap top,Vest top,Garment Upper body,Solid,White,Light,White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,Strap top (1),Vest top,Garment Upper body,Stripe,Off White,Dusty Light,White,Jersey Basic,Ladieswear,Ladieswear,Womens Everyday Basics,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,OP T-shirt (Idro),Bra,Underwear,Solid,Black,Dark,Black,Clean Lingerie,Lingeries/Tights,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,OP T-shirt (Idro),Bra,Underwear,Solid,White,Light,White,Clean Lingerie,Lingeries/Tights,Ladieswear,Womens Lingerie,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [ ]:
articles_df.describe()

In [ ]:
len(articles_df.columns)

In [ ]:
# iterating the columns
for col in articles_df.columns:
    print(col)

### Describe for each column

In [ ]:
len(articles_df['article_id'].unique())

### Retrived only needed columns

In [3]:
articles_df.drop(['product_code', 'product_type_no', 'graphical_appearance_no', 'colour_group_code', 'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no', 'index_code',  'index_group_no', 'section_no',  'garment_group_no'  ], axis=1, inplace=True)

In [ ]:
articles_df.head()

## EDA for transactions_train.csv

In [4]:
transactions_train_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv")
transactions_train_df.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [ ]:
transactions_train_df['t_dat'].unique()

In [ ]:
len(transactions_train_df)

### Convert to datetime column

In [5]:
transactions_train_df['t_dat'] = pd.to_datetime(transactions_train_df['t_dat'])

### CPU can't load and process the whole of DF, so 2018 year data has retrieved

In [ ]:
df_filtered = transactions_train_df[transactions_train_df['t_dat'].dt.strftime('%Y') == '2018']

In [ ]:
len(df_filtered)

### Deal with SettingWithCopyWarning in Pandas

In [7]:
pd.options.mode.chained_assignment = None  # default='warn'

### Drop uncessary columns

In [8]:
df_filtered.drop(['t_dat', 'price', 'sales_channel_id'], axis=1, inplace=True)

In [ ]:
df_filtered.head()

In [ ]:
len(df_filtered)

### Merge DF (articles_df + df_filtered) on article_id column

In [9]:
merged_df = pd.merge(articles_df, df_filtered, how='right', on = 'article_id')

In [ ]:
len(merged_df)

In [ ]:
merged_df.head()

In [ ]:
merged_df.duplicated()

In [10]:
merged_df.drop_duplicates(inplace=True)

In [11]:
merged_df.dropna(inplace=True)

In [ ]:
len(merged_df)

In [ ]:
merged_df.head()

### Merged all content columns into one ('detail_desc')

In [ ]:
#merged_df['detail_desc'] = merged_df['detail_desc']+merged_df['prod_name']+merged_df['product_type_name']+merged_df['product_group_name']+merged_df['graphical_appearance_name']+merged_df['colour_group_name']+merged_df['perceived_colour_value_name']+merged_df['perceived_colour_master_name']+merged_df['department_name']+merged_df['index_name']+merged_df['index_group_name']+merged_df['section_name']+merged_df['garment_group_name']

In [ ]:
merged_df['detail_desc'] = merged_df['detail_desc']+merged_df['prod_name']+merged_df['product_type_name']+merged_df['product_group_name']+merged_df['graphical_appearance_name']+merged_df['colour_group_name']+merged_df['department_name']+merged_df['index_name']+merged_df['garment_group_name']

In [ ]:
merged_df.columns

### Drop uncessary columns

In [12]:
merged_df.drop(['prod_name', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name', 'index_name', 'index_group_name', 'section_name', 'garment_group_name'],axis=1,inplace=True )

In [69]:
merged_df.head()

,article_id,detail_desc,customer_id
0,663713001,"Lace push-up body with underwired, moulded, pa...",000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
1,541518023,"Lace push-up bras with underwired, moulded, pa...",000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
2,505221004,Jumper in rib-knit cotton with hard-worn detai...,00007d2de826758b65a93dd24ce629ed66842531df6699...
3,685687003,V-neck knitted jumper with long sleeves and ri...,00007d2de826758b65a93dd24ce629ed66842531df6699...
4,685687004,V-neck knitted jumper with long sleeves and ri...,00007d2de826758b65a93dd24ce629ed66842531df6699...


In [ ]:
len(merged_df)

In [ ]:
len(merged_df['article_id'].unique())

In [13]:
unique_article_df = merged_df[['article_id', 'detail_desc']]

In [ ]:
len(unique_article_df)

### Merge row data with same product_code

In [14]:
unique_article_df= unique_article_df.assign(detail_desc=unique_article_df.groupby('article_id')['detail_desc'].ffill()).drop_duplicates('article_id', keep='last')
#unique_article_df= unique_article_df.drop_duplicates('article_id', keep='last')

In [15]:
len(unique_article_df)

41797

In [ ]:
unique_article_df['detail_desc']

### Load Universal Sentence Encoder Model (v4)

In [16]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

2022-02-22 05:44:56.496903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-22 05:44:56.586282: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-22 05:44:56.587337: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-22 05:44:56.589350: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

### Create Embedding

In [17]:
embeddings = embed(unique_article_df['detail_desc'].tolist())

print(embeddings)

tf.Tensor(
[[-0.02468695  0.06708961 -0.05027578 ... -0.0103434  -0.01386723
   0.01119992]
 [ 0.01308781  0.04737314 -0.0252722  ... -0.00014595 -0.06259475
   0.0557929 ]
 [ 0.05388185  0.03516567  0.03573162 ...  0.05809964  0.0166113
   0.02516435]
 ...
 [ 0.00210165  0.03172468 -0.06208596 ...  0.01425873 -0.06932767
  -0.00720803]
 [-0.03853679  0.03271476  0.02007107 ... -0.05753789 -0.07531829
   0.00097143]
 [ 0.01678361  0.03238354 -0.06565922 ...  0.02641837 -0.01156777
   0.03209247]], shape=(41797, 512), dtype=float32)


In [ ]:
embeddings.shape

### Retrieved unique product_code

In [18]:
article_id = set(unique_article_df['article_id'])

In [25]:
unique_article_id_list = list(article_id)

### Create Cosine Similarity Matrix using sklearn library

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
sim = cosine_similarity(embeddings)

### Create DataFrame by setting product_code as DF index

In [30]:
sim = pd.DataFrame(data=sim,columns=unique_article_id_list,index=unique_article_id_list)
sim.head()

,638976001,638976002,522453001,522453002,637141001,637141005,637141006,651690001,700842002,651690003,...,685417002,593494001,593494002,642646003,593494003,527958005,642646001,527958001,527958006,657195001
638976001,1.000000,0.433151,0.173133,0.551032,0.580429,0.355543,0.571271,0.513673,0.535189,0.599634,...,0.387224,0.601640,0.512051,0.567571,0.568590,0.490689,0.398355,0.462707,0.421647,0.270264
638976002,0.433151,1.000000,0.156156,0.384014,0.569213,0.591850,0.736383,0.589489,0.547066,0.532574,...,0.615123,0.505298,0.476532,0.617805,0.509730,0.564811,0.362034,0.511418,0.543797,0.429657
522453001,0.173133,0.156156,1.000000,0.195702,0.196946,0.270621,0.150274,0.212961,0.300220,0.152426,...,0.265418,0.146080,0.251473,0.096179,0.078968,0.154765,0.169928,0.144451,0.158933,0.192042
522453002,0.551032,0.384014,0.195702,1.000000,0.475733,0.371688,0.433331,0.545669,0.411702,0.552360,...,0.397211,0.475515,0.379044,0.474261,0.543321,0.468548,0.511121,0.338017,0.348953,0.363551
637141001,0.580429,0.569213,0.196946,0.475733,1.000000,0.494349,0.650908,0.559571,0.519563,0.624803,...,0.484879,0.479547,0.528897,0.706137,0.817412,0.683223,0.334491,0.402266,0.536606,0.415656


In [ ]:
sim.index

### User-defined function to find top 12 similar articles

In [31]:
def recommended_articles(article_id): 
    show_cos_sim = sim.loc[article_id]
    rank = np.argsort(show_cos_sim.values)[::-1]
    arr= rank[1:13]
    arr=arr.flatten()
    return ' '.join(map(str, arr.tolist()))

### For each artilce, find related recommend items

In [33]:
recommended_list = {}
for item in unique_article_id_list:
    recommended_list[item] = recommended_articles(item)

In [34]:
len(merged_df), len(recommended_list)

(3829403, 99)

### Retrieved Entries which product code contained in training data (product_code)

In [35]:
# article_ids =merged_df[merged_df['article_id'].isin(article_id)][['article_id', 'customer_id']]
article_ids =merged_df[merged_df['article_id'].isin(unique_article_id_list)][['article_id', 'customer_id']]

In [36]:
len(article_ids)

7312

In [37]:
article_ids.drop_duplicates(inplace=True)

In [38]:
article_ids.dropna(inplace=True)

In [41]:
len(article_ids)

7312

In [39]:
article_ids.rename(columns={'article_id': 'prediction'}, inplace=True)

### ReMap pandas data values in DF column

In [42]:
article_ids.replace({"prediction": recommended_list}, inplace=True)

In [55]:
article_ids.head()

,prediction,customer_id
1186,32371 19579,06884807549bd7463a0f21e97beacbd0bd28c01dfde80d...
1201,16882 28325,06a22599668b2dce074dfd55d8e698b8497f0eb1411bb7...
1396,4150 21240,074d84350aa83aceda0ebabb9ac152322a044ed47bca9f...
1907,5 37423,0a723e885fb15d2cdd76df0a0819423c6ea5f2c6860931...
1909,26366 332,0a723e885fb15d2cdd76df0a0819423c6ea5f2c6860931...


In [45]:
len(article_ids)

7312

## EDA for customers.csv

In [46]:
customer_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv")
customer_df.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [ ]:
len(customer_df['customer_id'].unique())

In [47]:
customer_df.drop(['FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age', 'postal_code'], axis=1, inplace=True)

In [56]:
customer_df.head()

,customer_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...


In [57]:
merged_article_df = pd.merge(article_ids, customer_df, how='right', on = 'customer_id')

In [58]:
merged_article_df.head()

,prediction,customer_id
0,NaN,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,NaN,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,NaN,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,NaN,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,NaN,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...


In [ ]:
merged_article_df.isna()

In [59]:
filled_data = '32371 19579'

In [54]:
filled_data

0          NaN
1          NaN
2          NaN
3          NaN
4          NaN
          ... 
1372661    NaN
1372662    NaN
1372663    NaN
1372664    NaN
1372665    NaN
Name: prediction, Length: 1372666, dtype: object

In [60]:
merged_article_df['prediction'].fillna(filled_data, inplace=True)

In [61]:
merged_article_df.isna()

,prediction,customer_id
0,False,False
1,False,False
2,False,False
3,False,False
4,False,False
...,...,...
1372661,False,False
1372662,False,False
1372663,False,False
1372664,False,False


In [ ]:
len(merged_article_df)

In [62]:
merged_article_df.to_csv("submission.csv", index=False)

## EDA for sample_submission.csv

In [63]:
sample_submission_df = pd.read_csv("/kaggle/input/h-and-m-personalized-fashion-recommendations/sample_submission.csv")
sample_submission_df.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [64]:
test=sample_submission_df['prediction'][0]

In [65]:
type(test)

str

In [66]:
arr = test.split()

In [67]:
len(arr)

12